In [1]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random


In [2]:
HTML("<script src='d3.js'></script>")


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/calls.csv')
data.shape

(14, 2)

In [7]:
data.head()


,operators,calls
0,Adam,31
1,Danilo,24
2,Luis,49
3,Mark,16
4,Joe,51


In [8]:
data_dict = data.to_dict(orient="records")
data_dict[:5]


[{'calls': 31, 'operators': 'Adam'},
 {'calls': 24, 'operators': 'Danilo'},
 {'calls': 49, 'operators': 'Luis'},
 {'calls': 16, 'operators': 'Mark'},
 {'calls': 51, 'operators': 'Joe'}]

In [9]:
js_text_template = Template('''
var margin = {
        top: 20,
        right: 20,
        bottom: 30,
        left: 40
    },
    width = 500 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

var x = d3.scale.ordinal()
    .rangeRoundBands([0, width], .1);

var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

var svg = d3.select("#graph-div").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var data = $data;

x.domain(data.map(function(d) {
    return d.salesperson;
}));
y.domain([0, d3.max(data, function(d) {
    return d.sales;
})]);

svg.append("g")
    .attr("class", "x axis")
    .attr("transform", "translate(0," + height + ")")
    .call(xAxis);

svg.append("g")
    .attr("class", "y axis")
    .call(yAxis);

svg.selectAll(".bar")
    .data(data)
    .enter().append("rect")
    .attr("class", "bar")
    .attr("x", function(d) {
        return x(d.salesperson);
    })
    .attr("width", x.rangeBand())
    .attr("y", function(d) {
        return y(d.sales);
    })
    .attr("height", function(d) {
        return height - y(d.sales);
    });

for (let i = 0; i < data.length-1; i++) {
    svg.append("line")
        .style("stroke", "lightgreen")
        .style("stroke-width", 3)
        .attr("x1", x(data[i].salesperson)+x.rangeBand()/2)
        .attr("y1", y(data[i].sales))
        .attr("x2", x(data[i+1].salesperson)+x.rangeBand()/2)
        .attr("y2", y(data[i+1].sales));
}
''');


In [11]:
html_template = Template('''
    <style>
    .bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}
    </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.substitute({
    "data":json.dumps(data_dict)
})

HTML(html_template.substitute({'js_text':js_text}))